In [ ]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

# Calculate energy using the given formula
def calculate_energy(image):
    gray = np.mean(image, axis=2)  # Convert to grayscale
    dx = np.zeros_like(gray)
    dy = np.zeros_like(gray)

    # Sobel-like filters for x and y gradients
    dx[:, 1:-1] = gray[:, 2:] - gray[:, :-2]
    dy[1:-1, :] = gray[2:, :] - gray[:-2, :]

    energy = np.abs(dx) + np.abs(dy)
    return energy

# Compute cumulative energy map
def compute_cumulative_energy(energy):
    rows, cols = energy.shape
    cum_energy = energy.copy()

    for i in range(1, rows):
        for j in range(cols):
            left = cum_energy[i - 1, j - 1] if j > 0 else float('inf')
            up = cum_energy[i - 1, j]
            right = cum_energy[i - 1, j + 1] if j < cols - 1 else float('inf')
            cum_energy[i, j] += min(left, up, right)

    return cum_energy

# Find the seam to remove
def find_seam(cum_energy):
    rows, cols = cum_energy.shape
    seam = np.zeros(rows, dtype=np.int32)
    seam[-1] = np.argmin(cum_energy[-1])

    for i in range(rows - 2, -1, -1):  # Corrected loop range
        prev_col = seam[i + 1]
        col_start = max(prev_col - 1, 0)
        col_end = min(prev_col + 2, cols)  # Avoid out-of-bound access
        offset = np.argmin(cum_energy[i, col_start:col_end])
        seam[i] = col_start + offset

    return seam

# Remove the seam from the image
def remove_seam(image, seam):
    rows, cols, _ = image.shape
    new_image = np.zeros((rows, cols - 1, 3), dtype=image.dtype)

    for i in range(rows):
        new_image[i, :, :] = np.delete(image[i, :, :], seam[i], axis=0)

    return new_image

# Visualize the seam
def visualize_seam(image, seam):
    image_with_seam = image.copy()
    for i in range(len(seam)):
        image_with_seam[i, seam[i]] = [255, 0, 0]  # Red for seam visualization
    return image_with_seam

# Main function to resize the image
def seam_carve(image, target_width, target_height):
    current_image = image.copy()
    seams_visualization = image.copy()

    # Resize width
    while current_image.shape[1] > target_width:
        energy = calculate_energy(current_image)
        cum_energy = compute_cumulative_energy(energy)
        seam = find_seam(cum_energy)
        seams_visualization = visualize_seam(seams_visualization, seam)
        current_image = remove_seam(current_image, seam)

    # Resize height (transpose approach)
    current_image = np.transpose(current_image, (1, 0, 2))
    seams_visualization = np.transpose(seams_visualization, (1, 0, 2))

    while current_image.shape[1] > target_height:
        energy = calculate_energy(current_image)
        cum_energy = compute_cumulative_energy(energy)
        seam = find_seam(cum_energy)
        seams_visualization = visualize_seam(seams_visualization, seam)
        current_image = remove_seam(current_image, seam)

    current_image = np.transpose(current_image, (1, 0, 2))
    seams_visualization = np.transpose(seams_visualization, (1, 0, 2))

    return current_image, seams_visualization

# Load input image
def main():
    input_name = input("PIC.jpg").strip()
    input_path = f"./{input_name}"  # Assumes the file is in the current directory
    try:
        input_image = np.array(Image.open(input_path))
    except FileNotFoundError:
        print("Error: File not found. Please check the name and try again.")
        return
    except PermissionError:
        print("Error: Permission denied. Please check the file permissions.")
        return

    target_width, target_height = input_image.shape[1] // 2, input_image.shape[0] // 2

    resized_image, seams_image = seam_carve(input_image, target_width, target_height)

    # Save results
    Image.fromarray(resized_image.astype(np.uint8)).save('resized_image.jpg')
    Image.fromarray(seams_image.astype(np.uint8)).save('seams_image.jpg')

    # Display results
    plt.figure(figsize=(15, 5))
    plt.subplot(1, 3, 1)
    plt.title('Input Image')
    plt.imshow(input_image)

    plt.subplot(1, 3, 2)
    plt.title('Resized Image')
    plt.imshow(resized_image)

    plt.subplot(1, 3, 3)
    plt.title('Seams Visualization')
    plt.imshow(seams_image)

    plt.show()

main()


PIC.jpgPIC.jpg
